In [ ]:
from mpi4py import MPI
from dolfinx import mesh

msh = mesh.create_rectangle(
    comm=MPI.COMM_WORLD,
    points=((0.0,0.0),(1.0,1.0)),
    n=(16,16),
    cell_type=mesh.CellType.triangle
)

V = fem.function("Lagrange",1)